**@author: James V. Talwar**

# LDpred2-auto

**About:** This notebook generates LDpred2-auto polygenic risk scores (PRSs) for given reference panels, summary statistics, and datasets (on which to compute PRSs) of interest, which here is for prostate cancer risk prediction in the UKBB and ELLIPSE.
 - For more details on the specifics of LDpred2 and its implementation both this [ldpred2 specific tutorial](https://privefl.github.io/bigsnpr/articles/LDpred2.html) and this [general PRS tutorial](https://choishingwan.github.io/PRS-Tutorial/ldpred/) may prove helpful.

In [1]:
# Load packages bigsnpr and bigstatsr
library(bigsnpr)
library(pROC)
options(bigstatsr.check.parallel.blas = FALSE)
options(default.nproc.blas = NULL)

Loading required package: bigstatsr

Type 'citation("pROC")' for a citation.


Attaching package: ‘pROC’


The following objects are masked from ‘package:stats’:

    cov, smooth, var




Load in summary statistics:

In [2]:
sumstats <- bigreadr::fread2("../../Data/grievous_harmonized/PC/Conti_SSF/GRIEVOUS_Formatted/MergedSSF.ssf") 

In [4]:
sumstats <- setNames(sumstats, c("index", "chr", "pos", "rsid", "a0", "a1", "p", "beta", "beta_se", "N")) # <-- need to customize SSF ordering/naming 

Generate correlation matrix from full ELLIPSE genotype dataset (as these make up the bulk of the original data in the above defined summary statistics):

In [8]:
fullPath <- "../../Data/LDpred2/PC/ELLIPSE_Reference_Panel/ELLIPSE_All/chr"

In [10]:
# Get maximum amount of cores
NCORES <- nb_cores()

# Define temporary directory/files for genetic maps and correlation matrix 
tmp <- "../../Data/temporary/tmp-data/corr" 
tmpGenomesPos <- "../../Data/temporary/tmp-genomes-position" 

# Initialize variables for storing the LD score and LD matrix
corr <- NULL
ld <- NULL
# We want to know the ordering of samples in the bed file 
info_snp <- NULL
fam.order <- NULL

for (chr in 1:21){ # <-- Ends at 21 as chrom 22 (HapMap3+) SNPs did not pass QC
    # preprocess the bed file (only need to do once for each data set)
    snp_readBed(paste0(fullPath, chr, ".bed")) 
    
    # now attach the genotype object
    obj.bigSNP <- snp_attach(paste0(fullPath, chr, ".rds"))
    
    # extract the SNP information from the genotype
    map <- obj.bigSNP$map[-3]
    names(map) <- c("chr", "rsid", "pos", "a0", "a1") #opposite classic implementation - as grievous process sets ALT allele to reference 
    map$chr <- as.character(map$chr)
    
    # perform SNP matching
    tmp_snp <- snp_match(sumstats[sumstats$chr==chr,], map)
    info_snp <- rbind(info_snp, tmp_snp)
    
    # Assign the genotype to a variable for easier downstream analysis
    genotype <- obj.bigSNP$genotypes
    
    # Rename the data structures
    CHR <- as.integer(map$chr)
    POS <- map$pos
    
    # get genetic maps and convert; downloads genetic maps available from 1000G to defined tmpGenomesPos
    POS2 <- snp_asGeneticPos(CHR, POS, dir = tmpGenomesPos)# <-- uses genetic maps to interpolate physical positions (bp) to genetic positions (in cM)
    
    # Extract SNPs that are included in the chromosome
    ind.chr <- which(tmp_snp$chr == chr)
    ind.chr2 <- tmp_snp$`_NUM_ID_`[ind.chr]
    
    # calculate LD matrices 
    corr0 <- snp_cor(genotype, ind.col = ind.chr2, ncores = NCORES, infos.pos = POS2[ind.chr2], size = 3 / 1000) #<-- use paper recommended window size of 3 cM.
    
    if (chr == 1) {
        ld <- Matrix::colSums(corr0^2)
        corr <- as_SFBM(corr0, tmp)
    } 
    else {
        ld <- c(ld, Matrix::colSums(corr0^2))
        corr$add_columns(corr0, nrow(corr))
    }
}

2,228,360 variants to be matched.

0 ambiguous SNPs have been removed.

77,938 variants have been matched; 0 were flipped and 0 were reversed.

2,396,154 variants to be matched.

0 ambiguous SNPs have been removed.

83,671 variants have been matched; 0 were flipped and 0 were reversed.

2,000,410 variants to be matched.

0 ambiguous SNPs have been removed.

70,872 variants have been matched; 0 were flipped and 0 were reversed.

2,019,285 variants to be matched.

0 ambiguous SNPs have been removed.

62,979 variants have been matched; 0 were flipped and 0 were reversed.

1,826,692 variants to be matched.

0 ambiguous SNPs have been removed.

62,988 variants have been matched; 0 were flipped and 0 were reversed.

1,797,783 variants to be matched.

0 ambiguous SNPs have been removed.

69,268 variants have been matched; 0 were flipped and 0 were reversed.

1,648,680 variants to be matched.

0 ambiguous SNPs have been removed.

54,374 variants have been matched; 0 were flipped and 0 were rev

Ensure no NA values in `ld` and confirm number of variants:

In [11]:
sum(is.na(ld))
length(ld)
dim(corr)

[1] 0

[1] 965472

[1] 965472 965472

Calculate heritability estimate (LD score regression):

In [12]:
df_beta <- info_snp[,c("beta", "beta_se", "N", "_NUM_ID_")]
ldsc <- snp_ldsc(ld, length(ld), chi2 = (df_beta$beta / df_beta$beta_se)^2, sample_size = df_beta$N, blocks = NULL)
h2_est <- ldsc[["h2"]]
h2_est

[1] 0.1285534

Load train, val, and test genotypes and ensure no misalignment with `info_snp`:

In [14]:
#Convert files to expected format:''
genotypeDirs = c("../../Data/LDpred2/PC/ELLIPSE_Reference_Panel/Train/",
                 "../../Data/LDpred2/PC/ELLIPSE_Reference_Panel/Val/",
                 "../../Data/LDpred2/PC/ELLIPSE_Reference_Panel/Test/")

for (datasetPath in genotypeDirs){
    print(paste0("Beginning ", tail(strsplit(datasetPath, split = "/")[[1]], 1)))
    datasetSpecific <- NULL
    for (chr in 1:21) { #<-- again only 21 as no 22 QC passed SNPs
        # preprocess the bed file (only need to do once for each data set)
        snp_readBed(paste0(datasetPath, "chr", chr, ".bed")) 
    
        # now attach the genotype object
        rSnpObject <- snp_attach(paste0(datasetPath, "chr", chr, ".rds"))
        
        # extract the SNP information from the genotype
        map <- rSnpObject$map[-3]
        names(map) <- c("chr", "rsid", "pos", "a0", "a1") #opposite classic implementation - as grievous process sets ALT allele to reference 
        map$chr <- as.character(map$chr)
        
        # perform SNP matching
        tmp_snp <- snp_match(sumstats[sumstats$chr==chr,], map)
        datasetSpecific <- rbind(datasetSpecific, tmp_snp)
    }
    
    print(paste0("Completed ", tail(strsplit(datasetPath, split = "/")[[1]], 1), "; STATUS: ", all(datasetSpecific == info_snp)))
}

[1] "Beginning Train"


2,228,360 variants to be matched.

0 ambiguous SNPs have been removed.

77,938 variants have been matched; 0 were flipped and 0 were reversed.

2,396,154 variants to be matched.

0 ambiguous SNPs have been removed.

83,671 variants have been matched; 0 were flipped and 0 were reversed.

2,000,410 variants to be matched.

0 ambiguous SNPs have been removed.

70,872 variants have been matched; 0 were flipped and 0 were reversed.

2,019,285 variants to be matched.

0 ambiguous SNPs have been removed.

62,979 variants have been matched; 0 were flipped and 0 were reversed.

1,826,692 variants to be matched.

0 ambiguous SNPs have been removed.

62,988 variants have been matched; 0 were flipped and 0 were reversed.

1,797,783 variants to be matched.

0 ambiguous SNPs have been removed.

69,268 variants have been matched; 0 were flipped and 0 were reversed.

1,648,680 variants to be matched.

0 ambiguous SNPs have been removed.

54,374 variants have been matched; 0 were flipped and 0 were rev

[1] "Completed Train; STATUS: TRUE"
[1] "Beginning Val"


2,228,360 variants to be matched.

0 ambiguous SNPs have been removed.

77,938 variants have been matched; 0 were flipped and 0 were reversed.

2,396,154 variants to be matched.

0 ambiguous SNPs have been removed.

83,671 variants have been matched; 0 were flipped and 0 were reversed.

2,000,410 variants to be matched.

0 ambiguous SNPs have been removed.

70,872 variants have been matched; 0 were flipped and 0 were reversed.

2,019,285 variants to be matched.

0 ambiguous SNPs have been removed.

62,979 variants have been matched; 0 were flipped and 0 were reversed.

1,826,692 variants to be matched.

0 ambiguous SNPs have been removed.

62,988 variants have been matched; 0 were flipped and 0 were reversed.

1,797,783 variants to be matched.

0 ambiguous SNPs have been removed.

69,268 variants have been matched; 0 were flipped and 0 were reversed.

1,648,680 variants to be matched.

0 ambiguous SNPs have been removed.

54,374 variants have been matched; 0 were flipped and 0 were rev

[1] "Completed Val; STATUS: TRUE"
[1] "Beginning Test"


2,228,360 variants to be matched.

0 ambiguous SNPs have been removed.

77,938 variants have been matched; 0 were flipped and 0 were reversed.

2,396,154 variants to be matched.

0 ambiguous SNPs have been removed.

83,671 variants have been matched; 0 were flipped and 0 were reversed.

2,000,410 variants to be matched.

0 ambiguous SNPs have been removed.

70,872 variants have been matched; 0 were flipped and 0 were reversed.

2,019,285 variants to be matched.

0 ambiguous SNPs have been removed.

62,979 variants have been matched; 0 were flipped and 0 were reversed.

1,826,692 variants to be matched.

0 ambiguous SNPs have been removed.

62,988 variants have been matched; 0 were flipped and 0 were reversed.

1,797,783 variants to be matched.

0 ambiguous SNPs have been removed.

69,268 variants have been matched; 0 were flipped and 0 were reversed.

1,648,680 variants to be matched.

0 ambiguous SNPs have been removed.

54,374 variants have been matched; 0 were flipped and 0 were rev

[1] "Completed Test; STATUS: TRUE"


**Initialize and run LDpred2-auto:**

In [16]:
names(df_beta) <- c("beta", "beta_se", "n_eff", "_NUM_ID_")

In [17]:
coef_shrink = 0.95 # <-- Initialize robust LDpred2-auto shrinkage coefficient to paper reported/tutorial value (0.95)

#set.seed(3)  # <-- uncomment to get the same result every time

multi_auto <- snp_ldpred2_auto(
    corr,
    df_beta,
    h2_init = h2_est,
    vec_p_init = seq_log(1e-4, 0.2, length.out = 30),
    ncores = NCORES, allow_jump_sign = FALSE, 
    use_MLE = FALSE,
    shrink_corr = coef_shrink
)

Filter out bad chains (according to recommended LDpred2 tutorial procedure):

In [18]:
(range <- sapply(multi_auto, function(auto) diff(range(auto$corr_est)))) #<-- range should be between 0 and 2

[1] 0.06009418 0.05971744 0.05963770 0.05986703 0.05928711 0.05301412
 [7] 0.05315460 0.05958733 0.05919477 0.05579027 0.05740740 0.05478845
[13] 0.05164020 0.05235011 0.05958013 0.05828277 0.05921756 0.05574234
[19] 0.05358231 0.05498547 0.05359661 0.05926878 0.05545104 0.05918333
[25] 0.05999581 0.05575276 0.05824498 0.05108063 0.06430708 0.06011483

In [19]:
(keep <- which(range > (0.95 * quantile(range, 0.95, na.rm = TRUE))))

[1]  1  2  3  4  5  8  9 11 15 16 17 22 24 25 27 29 30

Get final LDpred2-auto effects using chains that passed filtering: 

In [20]:
beta_auto <- rowMeans(sapply(multi_auto[keep], function(auto) auto$beta_est))

Generate test set (UKBB) LDpred2-auto predictions:

In [21]:
testDir <- genotypeDirs[3]
test_auto <- NULL

for(chr in 1:21){
    print(chr)
    rSnpObject <- snp_attach(paste0(testDir, "chr", chr, ".rds"))
    genotypeWithNAs <- rSnpObject$genotypes
    genotype <- snp_fastImputeSimple(genotypeWithNAs, method = "mean2", ncores = 8) #

    #Calc PRS for all samples
    ind.test <- 1:nrow(genotype) 
    
    # Extract SNPs in this chromosome
    chr.idx <- which(info_snp$chr == chr)
    ind.chr <- info_snp$`_NUM_ID_`[chr.idx]
    
    tmp <- big_prodVec(genotype, 
                       beta_auto[chr.idx]) # chr.idx: the row elements of info_snp which are in order; Num_ID reset's the values which restart at each chromosome...

    if(is.null(test_auto)){
        test_auto <- tmp
    }
    
    else{
        test_auto <- test_auto + tmp
    }
   
}

[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21


Load in test set phenotype information to generate LDpred2-auto (comprehensive) prediction summary for performance evaluation:

In [23]:
ukbbPhenoPath <- "../../Data/PhenosAndIDs/PC/ukb.ellipse.pheno.tsv"
ukbbPheno <- read.table(ukbbPhenoPath)
names(ukbbPheno) <- c("IID", "AGE", "SEX", "PHENOTYPE","ANCESTRY")
ukbbPheno$PHENOTYPE <- ukbbPheno$PHENOTYPE - 1

Ensure predictions for each individual align with observed phenotype:

In [24]:
rSnpObject <- snp_attach(paste0(testDir, "chr1.rds"))
testIDs <- rSnpObject$fam$sample.ID
testPheno <- ukbbPheno[ukbbPheno$IID %in% testIDs,]
testFamOrder <- match(testIDs, testPheno$IID)
testPhenoOrdered <- testPheno[testFamOrder, ]

In [25]:
testSetScores <- testPhenoOrdered[,c("IID", "PHENOTYPE")]
testSetScores <- data.frame(testSetScores, NewValue = test_auto)
names(testSetScores) <- c("IID", "Labels", "Predictions")
testSetScores <- testSetScores[, c(1,3,2)]

In [ ]:
#write.table(testSetScores, file = "../../../Predictions/PC/LDpred2_auto/UKBB_Test_ELLIPSE_ALL_REF_Conti_SSF.tsv ", sep = "\t", row.names = FALSE, quote = FALSE)